In [1]:
%load_ext line_profiler

In [2]:
from collections import Counter
from functools import partial

import numpy as np
rng = np.random.default_rng()
np.set_printoptions(suppress=True, linewidth=180, edgeitems=5)
from tqdm.notebook import tqdm, trange
from tqdm.contrib.concurrent import process_map

from scipy.stats import rankdata
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings

In [111]:
# ns = [0,10,10,11,12,13,14,15,16,17]
# ns = [0,5,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
# ns = [0]


students = np.logspace(-1, 1, 21)
display(students.round(4))

# ns = [0,5,5,6,7,8,9,10,11,12,13,14,15]
# ms = 5
# # arrs = np.array([[20,0,0,0,0],
# #                  [19,19,0,0,0],
# #                  [18,18,18,0,0],
# #                  [17,17,17,17,0],
# #                  [16,16,16,16,16],])
# arrs = np.array([[20,0,0,0,0],
#                  [15,16,17,18,19],
#                  [0,0,0,0,0],
#                  [0,0,0,0,0],
#                  [0,0,0,0,0],
#                  # [-1000,-1000,-1000,-1000,-1000],
#                  # [-1000,-1000,-1000,-1000,-1000],
#                  # [-1000,-1000,-1000,-1000,-1000],
#                 ])
# arrs = arrs / 20
# display(arrs)
# display(arrs.sum().round(2))


ns = [0,5,5,6,7,8,9,10,11,12,13,14,15]
ms = 10
arrs = np.array([[20,0,0,0,0,0,0,0,0,0],
                 [10,11,12,13,14,15,16,17,18,19],
                 [0,0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,0,0,0],
                ])
arrs = arrs / 20
display(arrs)
display(arrs.sum().round(2))
                 
                 

# ms = 4
# arrs = np.triu(np.tile(np.arange(31, 35),(4,1)).T) - 20
# display(arrs)
# display(arrs.sum())


# ns = [0,5,5,6,7,8,9,10,11,12,13,14,15]
# ms = 2
# arrs = np.array([[1,0],[.99,.99]])
# display(arrs)
# display(arrs.sum().round(2))


ranked = False

array([ 0.1   ,  0.1259,  0.1585,  0.1995,  0.2512,  0.3162,  0.3981,  0.5012,  0.631 ,  0.7943,  1.    ,  1.2589,  1.5849,  1.9953,  2.5119,  3.1623,  3.9811,  5.0119,  6.3096,
        7.9433, 10.    ])

array([[1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

8.25

In [112]:
def make_model(num_nodes=128, num_dense=4):
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
    
    inputs = tf.keras.layers.Input(shape=(ms*ms + ms + 1))
    x = tf.keras.layers.Flatten()(inputs)
    for _ in range(num_dense):
        x = tf.keras.layers.Dense(num_nodes, activation='relu')(x)
    output1 = tf.keras.layers.Dense(ms, name='Y0')(x)
    output1 = tf.keras.layers.Softmax()(output1)
    model = tf.keras.models.Model(inputs=inputs, outputs=output1)
    
    opt = tf.keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1)
    # opt = tf.keras.optimizers.SGD()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    return model

def softmax(x, b, axis=-1):
    x = x - np.max(x, axis=axis, keepdims=True)
    return np.exp(x/b) / np.exp(x/b).sum(axis=axis, keepdims=True)

# def softmax(x, b, axis=-1):
#     x = x - np.max(x)
#     with np.errstate(divide='ignore', invalid='ignore'):
#         res = np.exp(x/b) / np.exp(x/b).sum(axis=-1, keepdims=True)
#         res = np.nan_to_num(res)
#     return res

def make_x():
    x = arrs
    
    # rand = np.random.random(arrs.shape)
    # rand = (rand - 0.5)
    # x = x + rand*0.0001
    
    x = rng.permuted(x, axis=-1)
    return x

def eval_model(model, n = 1, disable=True):
    rewards = []
    branch_rewards = []
    for _ in trange(n, disable=disable):
        x = make_x()
        x_tree = np.concatenate([np.eye(ms+1).astype(int), np.tile(x.flatten(), (ms+1,1))], axis=1)
        y_pred = model(x_tree)
        branch_reward = (x_tree[0,ms+1:].reshape(ms,ms) * y_pred[1:].numpy()).sum(axis=1)
        rewards.append(y_pred[0].numpy())
        # reward = (y_pred[0] * branch_reward).numpy().sum()
        # rewards.append(reward)
        branch_rewards.append(branch_reward)
    return np.array(rewards), np.array(branch_rewards)

def make_data(b, n=1, disable=True, rand=False):
    X, Y = [], []
    for _ in trange(n, disable=disable):
        
        if ranked:
            x = make_x()
            r1 = softmax(rankdata(x,axis=1), b)
            r1sum = (r1*x).sum(axis=1)
            r0 = softmax(rankdata(r1sum), b)
                    
        else:
            x = make_x()
            r1 = softmax(x, b)
            r1sum = (r1*x).sum(axis=1)
            r0 = softmax(r1sum, b)
        
        pos = [0] * (ms+1)
        pos[0] = 1
        x0 = pos + list(x.flatten())
        y0 = rng.choice(np.arange(ms), p=r0)
        # while y0 in [2,3,4]:
        #     y0 = rng.choice(np.arange(ms), p=r0)
            
        
        pos = [0] * (ms+1)
        pos[y0+1] = 1
        x1 = pos + list(x.flatten())
        y1 = rng.choice(np.arange(ms), p=r1[y0])

        X.append(x0)
        Y.append(y0)
        X.append(x1)
        Y.append(y1)
    
    X = np.array(X)
    Y = np.array(Y)
    
    return X,Y

In [113]:
%%time
for b in students:
    r0s = []
    r1s = []
    for _ in trange(10000, disable=True):
        x = make_x()
        
        if ranked:
            r1 = (softmax(rankdata(x,axis=1), b) * x).sum(axis=1)
            r0 = (softmax(rankdata(r1), b) * r1).sum()
            r1s.append(r1)
            r0s.append(r0)
            
        else:
            r1 = (softmax(x, b) * x).sum(axis=1)
            r0 = (softmax(r1, b) * r1).sum()
            r1s.append(r1)
            r0s.append(r0)
    
        
    r1s = np.array(r1s)
    r0s = np.array(r0s)
    print(f"{b:.3f}, {r0s.mean():.3f}, {r1s.mean(axis=0).round(3)}, {softmax(r1s.mean(axis=0), b).round(3)}")

0.100, 0.971, [1.    0.876 0.    0.    0.    0.    0.    0.    0.    0.   ], [0.774 0.226 0.    0.    0.    0.    0.    0.    0.    0.   ]
0.126, 0.960, [0.997 0.857 0.    0.    0.    0.    0.    0.    0.    0.   ], [0.75  0.247 0.    0.    0.    0.    0.    0.    0.    0.   ]
0.158, 0.932, [0.984 0.838 0.    0.    0.    0.    0.    0.    0.    0.   ], [0.708 0.281 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001]
0.200, 0.860, [0.943 0.819 0.    0.    0.    0.    0.    0.    0.    0.   ], [0.623 0.333 0.006 0.006 0.006 0.006 0.006 0.006 0.006 0.006]
0.251, 0.726, [0.856 0.802 0.    0.    0.    0.    0.    0.    0.    0.   ], [0.483 0.389 0.016 0.016 0.016 0.016 0.016 0.016 0.016 0.016]
0.316, 0.556, [ 0.724  0.788  0.     0.     0.    -0.     0.     0.     0.     0.   ], [0.33  0.403 0.033 0.033 0.033 0.033 0.033 0.033 0.033 0.033]
0.398, 0.410, [0.578 0.776 0.    0.    0.    0.    0.    0.    0.    0.   ], [0.221 0.364 0.052 0.052 0.052 0.052 0.052 0.052 0.052 0.052]
0.501, 0.307, [ 0

In [114]:
def train_model(student):
    verbose = False
    if student == 1:
        print(f'student == {student:.3f}')
        verbose = True
    model = make_model()
    Xtrain, Ytrain = make_data(student, 2**16, disable=not verbose)
    model.fit(Xtrain, Ytrain, verbose=verbose)
    reward = eval_model(model, 10000, disable=not verbose)
    model.save(f'models/starting_{student:.3f}.keras')
    return reward

In [115]:
%%time
rewards = process_map(train_model, students, disable=True, max_workers=24)

student == 1.000


  0%|          | 0/65536 [00:00<?, ?it/s]

4096/4096 [==============================] - 60s 14ms/step - loss: 2.2862 - accuracy: 0.1441


  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 1.99 s, sys: 1.93 s, total: 3.91 s
Wall time: 3min 35s


In [116]:
for r, s in zip(rewards, students):
    
    res = r[0]*r[1]
    res = res.sum(axis=1).mean()
    
    print(f"{s:.3f} {res}")

0.100 0.9435386581597919
0.126 0.934904249909741
0.158 0.9130358712629908
0.200 0.8452128567467394
0.251 0.7127959726038939
0.316 0.495723955729507
0.398 0.36083794153754295
0.501 0.27229067992838846
0.631 0.19991475702329936
0.794 0.16806470262618517
1.000 0.13281195903902443
1.259 0.12565971708384582
1.585 0.11950420011338907
1.995 0.09398886449093591
2.512 0.09280471236817688
3.162 0.09365580401518081
3.981 0.08826207297499962
5.012 0.08956618279234406
6.310 0.0898381069543952
7.943 0.08348170598827391
10.000 0.08423783596700447


# Graphs

In [ ]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
tf.get_logger().setLevel('ERROR')
model2 = tf.keras.models.load_model(f"models/starting_{0.100:.3f}.keras")
%timeit r0, r1 = eval_model(model2, 1)

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()

    def on_batch_end(self, batch, logs=None):
        r0, r1 = eval_model(self.model, 10)
        r = (r0*r1).sum(axis=1).mean()
        
        
        try:
            self.model.rewards.append(r)
            self.model.accs.append(logs['accuracy'])
        except:
            self.model.rewards = [r]
            self.model.accs = [logs['accuracy']]

#         # Add the output to the logs
#         if logs is not None:
#             logs['reward'] = r

In [ ]:
models = {}

In [ ]:
%%time

with open('tree.log', 'a') as f:
    for student in students:
        
        print(f'student {student:.3f}', file=f, flush=True)

        models[student] = {}

        for teacher in students[::2][3:10]:

            print(f'teacher {teacher:.3f}', file=f, flush=True)
            models[student][teacher] = []
            for i in range(5):
                print(f'model {i}', file=f, flush=True)
                # model = make_model()
                model = tf.keras.models.load_model(f"models/starting_{student:.3f}.keras")

                opt = tf.keras.optimizers.Adam()
                loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
                model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

                X, Y = currs[f'{teacher:.3f}']
                model.fit(X[:2**17], Y[:2**17], callbacks=[CustomCallback()], verbose=False)
                # model.fit(X[:2**17], Y[:2**17], callbacks=[CustomCallback()])

                models[student][teacher].append(model)


        #         break
        #     break
        # break

In [ ]:
for student, s_models in models.items():
    # print(student, list(s_models))
    df = {}
    for teacher, model in s_models.items():
        kernel_size = 20
        kernel = np.ones(kernel_size) / kernel_size
        rewards = np.array([m.rewards for m in model]).mean(axis=0)
        rewards = np.convolve(rewards, kernel, mode='valid')
        df[f"{teacher:.3f}"] = rewards
    df = pd.DataFrame(df)
    
    print(f'Student {student:.3f}')
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        ax = sns.lineplot(df[[c for c in df.columns if float(c) >= student]])
        sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        plt.show()
    print()

In [ ]:
df[[c for c in df.columns if float(c) >= 1]]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    ax = sns.lineplot(df)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
1+1

In [ ]:
ax = sns.lineplot(df[df.columns[4:9]])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
students[::2][4:9]

# Eval Training

In [117]:
def eval_training(student, teacher, verbose=False):
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)

    
    if student == -1:
        model2 = make_model()
    else:
        model2 = tf.keras.models.load_model(f"models/starting_{student:.3f}.keras")
    # opt = tf.keras.optimizers.SGD()
    opt = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model2.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    nsum = 0
    
    X, Y = currs[f"{teacher:.3f}"]
    perm = np.random.permutation(len(X))
    X, Y = X[perm], Y[perm]
    
    rewards = []
    for i in ns:
        if i != 0:
            n = 2**i
            # X, Y = make_data(teacher, n, disable=True)
                        
            model2.fit(X[nsum:nsum+n], Y[nsum:nsum+n], verbose=False)
            nsum += n
            
        r0, r1 = eval_model(model2, 10)
        reward = (r0*r1).sum(axis=1).mean()
        rewards.append(reward)
        # print(f"{nsum}    ", end='\r')
    return rewards

In [118]:
def exp(student, verbose=True, n=20):
    # teachers = np.arange(student, 0.40001, 0.02)
    
    teachers = students
    
    t_ = list(teachers) * n
    s_ = [student for _ in t_]
    
    res = process_map(eval_training, s_, t_, max_workers=5, chunksize=1)
    
    return np.array(res)

In [119]:
%%time
ntotal = (2**np.array(ns[1:])).sum()

currs = process_map(make_data, students, [ntotal]*len(students), max_workers=6, chunksize=1)
currs = dict(zip([f"{s:.3f}" for s in students], currs))

  0%|          | 0/21 [00:00<?, ?it/s]

CPU times: user 906 ms, sys: 2.18 s, total: 3.09 s
Wall time: 46.4 s


In [120]:
%%time
exp(-1, n=1).reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/21 [00:00<?, ?it/s]

CPU times: user 52.2 ms, sys: 94.4 ms, total: 147 ms
Wall time: 1min 16s


array([[0.09, 0.09, 0.09, 0.1 , 0.14, 0.21, 0.23, 0.43, 0.84, 0.91, 0.95, 0.94, 0.94],
       [0.09, 0.09, 0.09, 0.1 , 0.14, 0.23, 0.25, 0.44, 0.8 , 0.92, 0.92, 0.92, 0.88],
       [0.08, 0.08, 0.08, 0.09, 0.1 , 0.18, 0.21, 0.45, 0.77, 0.88, 0.86, 0.91, 0.87],
       [0.08, 0.08, 0.08, 0.09, 0.1 , 0.18, 0.25, 0.34, 0.6 , 0.81, 0.85, 0.81, 0.83],
       [0.09, 0.1 , 0.11, 0.12, 0.18, 0.3 , 0.37, 0.36, 0.5 , 0.63, 0.73, 0.78, 0.7 ],
       [0.1 , 0.11, 0.12, 0.13, 0.19, 0.25, 0.32, 0.31, 0.32, 0.45, 0.45, 0.48, 0.5 ],
       [0.08, 0.09, 0.09, 0.1 , 0.13, 0.25, 0.23, 0.25, 0.24, 0.32, 0.32, 0.36, 0.4 ],
       [0.08, 0.09, 0.09, 0.1 , 0.11, 0.18, 0.16, 0.21, 0.24, 0.27, 0.29, 0.26, 0.26],
       [0.08, 0.09, 0.09, 0.1 , 0.11, 0.17, 0.14, 0.17, 0.21, 0.18, 0.21, 0.24, 0.21],
       [0.09, 0.09, 0.1 , 0.1 , 0.11, 0.14, 0.13, 0.14, 0.13, 0.16, 0.16, 0.16, 0.18],
       [0.08, 0.09, 0.1 , 0.11, 0.12, 0.11, 0.12, 0.14, 0.15, 0.13, 0.15, 0.14, 0.15],
       [0.08, 0.09, 0.09, 0.1 , 0.1 , 0.1 ,

In [121]:
%%time
exp(1, n=2).reshape(-1, len(students), len(ns)).mean(axis=0).round(3)

  0%|          | 0/42 [00:00<?, ?it/s]

CPU times: user 64.3 ms, sys: 120 ms, total: 184 ms
Wall time: 2min 20s


array([[0.134, 0.138, 0.138, 0.15 , 0.178, 0.249, 0.452, 0.482, 0.81 , 0.922, 0.935, 0.927, 0.942],
       [0.134, 0.137, 0.138, 0.15 , 0.179, 0.256, 0.463, 0.471, 0.781, 0.923, 0.932, 0.925, 0.915],
       [0.134, 0.137, 0.138, 0.15 , 0.178, 0.258, 0.48 , 0.463, 0.747, 0.889, 0.875, 0.898, 0.899],
       [0.134, 0.138, 0.139, 0.152, 0.182, 0.272, 0.488, 0.458, 0.669, 0.825, 0.823, 0.836, 0.828],
       [0.133, 0.137, 0.142, 0.151, 0.18 , 0.275, 0.463, 0.432, 0.493, 0.625, 0.67 , 0.683, 0.647],
       [0.132, 0.135, 0.146, 0.15 , 0.182, 0.262, 0.372, 0.392, 0.372, 0.452, 0.526, 0.522, 0.535],
       [0.132, 0.135, 0.146, 0.15 , 0.179, 0.261, 0.318, 0.33 , 0.306, 0.331, 0.324, 0.356, 0.387],
       [0.132, 0.135, 0.146, 0.149, 0.171, 0.229, 0.254, 0.25 , 0.241, 0.224, 0.24 , 0.269, 0.284],
       [0.132, 0.135, 0.145, 0.147, 0.164, 0.189, 0.205, 0.193, 0.217, 0.204, 0.206, 0.207, 0.225],
       [0.132, 0.136, 0.141, 0.144, 0.15 , 0.166, 0.182, 0.17 , 0.198, 0.169, 0.188, 0.163, 0.176],


In [122]:
%%time
exp(10, n=2).reshape(-1, len(students), len(ns)).mean(axis=0).round(3)

  0%|          | 0/42 [00:00<?, ?it/s]

CPU times: user 72.7 ms, sys: 112 ms, total: 185 ms
Wall time: 2min 20s


array([[0.084, 0.084, 0.084, 0.085, 0.085, 0.088, 0.102, 0.214, 0.305, 0.737, 0.91 , 0.928, 0.93 ],
       [0.084, 0.084, 0.084, 0.085, 0.085, 0.088, 0.102, 0.237, 0.314, 0.772, 0.914, 0.917, 0.93 ],
       [0.084, 0.084, 0.085, 0.085, 0.086, 0.088, 0.103, 0.275, 0.343, 0.684, 0.877, 0.897, 0.904],
       [0.084, 0.084, 0.085, 0.085, 0.086, 0.088, 0.104, 0.301, 0.356, 0.609, 0.788, 0.833, 0.829],
       [0.084, 0.084, 0.085, 0.085, 0.086, 0.088, 0.107, 0.276, 0.329, 0.419, 0.638, 0.656, 0.695],
       [0.084, 0.084, 0.085, 0.085, 0.086, 0.088, 0.113, 0.26 , 0.305, 0.316, 0.45 , 0.516, 0.545],
       [0.084, 0.084, 0.085, 0.085, 0.086, 0.088, 0.1  , 0.235, 0.314, 0.276, 0.273, 0.354, 0.374],
       [0.084, 0.084, 0.085, 0.085, 0.085, 0.087, 0.094, 0.19 , 0.247, 0.252, 0.244, 0.221, 0.249],
       [0.084, 0.084, 0.085, 0.085, 0.085, 0.086, 0.093, 0.141, 0.175, 0.2  , 0.211, 0.204, 0.175],
       [0.084, 0.084, 0.085, 0.085, 0.085, 0.086, 0.089, 0.112, 0.137, 0.173, 0.171, 0.194, 0.177],


In [50]:
X, Y = make_data(1, 100)

In [51]:
Y[::2]

array([0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1])

In [21]:
_18[:13,3:9].T-0.522

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.001,  0.001,  0.   ,  0.   ],
       [-0.001, -0.001, -0.001, -0.001,  0.   ,  0.   ,  0.   ,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001],
       [-0.003, -0.003, -0.002, -0.002, -0.001,  0.   ,  0.   ,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001],
       [-0.019, -0.011, -0.006, -0.004, -0.002, -0.001,  0.001,  0.001,  0.002,  0.002,  0.003,  0.003,  0.002],
       [-0.081, -0.064, -0.052, -0.033, -0.012, -0.003,  0.001,  0.003,  0.004,  0.004,  0.005,  0.006,  0.003],
       [-0.081, -0.076, -0.053, -0.035, -0.016, -0.001,  0.002,  0.006,  0.01 ,  0.009,  0.01 ,  0.006,  0.007]])

In [25]:
_18 - 0.522

array([[ 0.   ,  0.   ,  0.   ,  0.   , -0.001, -0.003, -0.019, -0.081, -0.081,  0.135,  0.284,  0.342,  0.382],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.001, -0.003, -0.011, -0.064, -0.076,  0.124,  0.256,  0.318,  0.372],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.001, -0.002, -0.006, -0.052, -0.053,  0.108,  0.251,  0.303,  0.355],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.001, -0.002, -0.004, -0.033, -0.035,  0.078,  0.236,  0.295,  0.342],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.001, -0.002, -0.012, -0.016,  0.047,  0.198,  0.256,  0.306],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.001, -0.003, -0.001,  0.029,  0.156,  0.225,  0.273],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.001,  0.001,  0.002,  0.014,  0.077,  0.178,  0.232],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.001,  0.001,  0.001,  0.003,  0.006,  0.015,  0.049,  0.126,  0.184],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.001,  0.001,  0.002,  0.004,  0.01 ,  0.016,  0.032, 

In [ ]:
df = pd.DataFrame(_264[:13,3:9].T - 0.227)
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    ax = sns.lineplot(df)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
_213[:10:2,:7] - 0.75

In [ ]:
%%time
exp(3.162, n=10).reshape(-1, len(students), len(ns)).mean(axis=0).round(3)

In [ ]:
[[1,0],
 [a,b]]
temp = 1

In [ ]:
def foo(x, y, temp):
    
    
    r1_H = np.exp(1/temp) / (np.exp(1/temp) + np.exp(0))
    r1_L = (x*np.exp(x/temp) + y*np.exp(y/temp)) / (np.exp(x/temp) + np.exp(y/temp))
    
    r_0 = (r1_H*np.exp(r1_H/temp) + r1_L*np.exp(r1_L/temp)) / (np.exp(r1_H/temp) + np.exp(r1_L/temp))
    
    branchL_prob = np.exp(r1_L/temp) / (np.exp(r1_H/temp) + np.exp(r1_L/temp))
    return branchL_prob

In [ ]:
space = np.logspace(-2, 2, 41)

In [ ]:
%%time
probs = {}
for x in np.linspace(0,1,101):
    for y in np.linspace(0,1,101):
        for temp in np.logspace(-2, 2, 81):
            temp = temp
            if 1>x>y>0:
                probs[x,y,temp] = foo(x,y,temp)

In [ ]:
max(probs, key=probs.get)

In [ ]:
probs[max(probs, key=probs.get)]

In [ ]:
{(x,y,t):v for (x,y,t),v in probs.items() if x==0.51 and y==0.50}